In [3]:
import os
os.chdir('/mnt/data/akshit.jain/visual_reco/mens_fashion_data')
! ls

attributes_meta.pk  products_meta.pk  user_clicks.pk


In [65]:
import pandas as pd
import numpy as np
from openrec.utils import Dataset
from openrec.utils.samplers import RandomPairwiseSampler
from openrec.recommenders import BPR
from openrec import ModelTrainer
from openrec.utils.evaluators import AUC
from openrec.utils.evaluators import Precision
from openrec.utils.samplers import EvaluationSampler

In [7]:
# clicks_df=pd.read_pickle('user_clicks.pk')
clicks_df.head()

,user_id,fsn,store
1,720e54266503706e11cbc692895b87b9,BRFER8N2FCYMMGVH,NaN
2,a1d5eadbcc129c2160bd5dd7c467a771,BRFER8N2FCYMMGVH,2oq/s9b/b1a/afm
3,7ea2ff595c07d0e78b0d7c845383006a,BRFER8N2FCYMMGVH,NaN
4,9c7597a3e6f00e1025193251fa11daa8,BRFER8N2FCYMMGVH,NaN
6,d5ae68001a8796f04649df3b46aab211,BRFER8N2FCYMMGVH,NaN


In [8]:
unique_users = pd.Series(clicks_df.user_id.unique()).to_frame('user_id')
unique_users['id']=unique_users.index
unique_fsns = pd.Series(clicks_df.fsn.unique()).to_frame('fsn')
unique_fsns['id']=unique_fsns.index

In [12]:
# express clicks and products interaction data as ints
denorm_df=pd.merge(clicks_df, unique_fsns, how='inner', on='fsn').merge(unique_users, on='user_id', suffixes=('_fsn', '_user'))
denorm_df.head()

,user_id,fsn,store,id_fsn,id_user
0,720e54266503706e11cbc692895b87b9,BRFER8N2FCYMMGVH,NaN,0,0
1,720e54266503706e11cbc692895b87b9,BRFEVYPNGATSTMGZ,NaN,20,0
2,720e54266503706e11cbc692895b87b9,BRFEVYPNURJYA99V,NaN,23,0
3,720e54266503706e11cbc692895b87b9,BRFEVYPNVFKHHHYZ,NaN,24,0
4,720e54266503706e11cbc692895b87b9,BRFER8N9PFPSCQTB,NaN,3672,0


In [15]:
final_df=denorm_df[['id_fsn','id_user']].sample(frac=1.0)

In [81]:
clicks_agg=final_df.groupby('id_user').count().reset_index()

In [85]:
clicks_agg[clicks_agg.id_fsn>1].id_fsn.sum()/clicks_agg.id_fsn.sum()

0.7423792431592888

In [61]:
TEST_FRAC=0.10
TRAIN_FRAC=1-TEST_FRAC
DATA_SIZE=final_df.shape[0]
TRAIN_SIZE=int(TRAIN_FRAC*DATA_SIZE)
TEST_SIZE=DATA_SIZE-TRAIN_SIZE
train_df = final_df[:TRAIN_SIZE]
test_df = val_df = final_df[TRAIN_SIZE:]

In [62]:
train_dat=np.asarray(train_df.rename(columns={'id_fsn':'item_id','id_user':'user_id'}).to_records(index=False))
test_dat=np.asarray(test_df.rename(columns={'id_fsn':'item_id','id_user':'user_id'}).to_records(index=False))
raw_data={
    'train_data': train_dat,
    'test_data': test_dat,
    'total_users': unique_users.shape[0],
    'total_items': unique_fsns.shape[0]
}

In [63]:
train_dataset = Dataset(raw_data['train_data'], raw_data['total_users'], raw_data['total_items'], name='Train')
test_dataset = Dataset(raw_data['test_data'], raw_data['total_users'], raw_data['total_items'], name='Test')

In [66]:
BATCH_SIZE=1000
DIM_EMBED = 50
TOTAL_ITER=int(1e4)
EVAL_ITER=TOTAL_ITER//10
train_sampler = RandomPairwiseSampler(batch_size=BATCH_SIZE, dataset=train_dataset, num_process=5)
val_sampler = EvaluationSampler(batch_size=BATCH_SIZE, dataset=test_dataset)

In [67]:
bpr_model = BPR(batch_size=BATCH_SIZE, total_users=train_dataset.total_users(), total_items=train_dataset.total_items(), 
                dim_user_embed=DIM_EMBED, dim_item_embed=DIM_EMBED, save_model_dir='bpr_recommender/', train=True, serve=True)

In [68]:
model_trainer = ModelTrainer(model=bpr_model)
auc_evaluator = AUC()
# precision_evaluator = Precision(precision_at=5)

In [73]:
model_trainer.train(total_iter=TOTAL_ITER, eval_iter=EVAL_ITER, save_iter=TOTAL_ITER, train_sampler=train_sampler, eval_samplers=[val_sampler], evaluators=[auc_evaluator])

[Training starts, total_iter: 10000, eval_iter: 1000, save_iter: 10000]
[iter 3000] loss: 139.601496
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from bpr_recommender/model.ckpt


Process _Sampler-56:
Process _Sampler-54:
Process _Sampler-58:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Process _Sampler-55:
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Process _Sampler-53:
  File "/mnt/data/virtualenv/tensorflow/lib/python3.5/site-packages/openrec/utils/samplers/sampler.py", line 14, in run
    self._q.put(input_npy, block=True)
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 82, in put
    if not self._sem.acquire(block, timeout):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/mnt/data/virtualenv/tensorflow/lib/python3.5/site-packages/openrec/utils/samplers/sampler.

KeyboardInterrupt
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()


KeyboardInterrupt: 

  File "/mnt/data/virtualenv/tensorflow/lib/python3.5/site-packages/openrec/utils/samplers/sampler.py", line 14, in run
    self._q.put(input_npy, block=True)
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 82, in put
    if not self._sem.acquire(block, timeout):
KeyboardInterrupt


In [71]:
! rm -r bpr_recommender